### HDS-M05: Deep Learning (Exercise III)

**Practical leader and prepared by:** Sharib Ali, PhD

[Setup instructions](https://github.com/sharibox/tutorial/blob/master/HDS-CDT_DL.md)

### Required packages

[1] [matplotlib](http://matplotlib.org) can be used for plotting graphs in python

[2] [pytorch](https://pytorch.org/docs/stable/index.html) is library widely used for bulding deep-learning frameworks

[3] [Tensorboard](https://pytorch.org/docs/stable/tensorboard.html) is used to visualise your training and validation loss and accuracy development - It is important to observe it!!!

[4] [TorchVision](https://pytorch.org/vision/stable/index.html) you can use available datasets that include one you are required to use in this tutorial (CIFAR10) and also use augmentations (meaning you can increase data variability for improved accuracy and model generalisation)

[5] [Scikit-learn] Metrics

**Reference:** 

[1] Original U-Net paper: https://arxiv.org/pdf/1505.04597.pdf

[2] Link to presentation: https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/


**What will you learn here?**

- You wil learn how to load your custom data for semantic segmentation task

- You will make an encoder-decoder U-Net architecture 

- You will learn how to group repeated blocks into one while designing U-Net

<img src="https://i.stack.imgur.com/EtyQs.png" style="width:800px;height:400px;">
<caption><center> <u>Figure</u>: U-Net architecture for image segmentation.</center></caption>

**Concentrate on:**

[1] Encoder layers (this is similar to your classification exercise)

[2] Decoder layers (upscaling of images occur here as you want to obtain per-pixel segmentation)

In [1]:
import torch
import torchvision 
from torch import nn
import numpy as np
# always check your version
print(torch.__version__)

1.10.0


#### New : We will setup a seed for reproducibility (ask tutor if you do not understand it)
- Setting a seed helps to regenerate the same result

In [2]:
import random
random_seed= 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True

### Data loading and pre-processing
**Steps**

[1] Load data - data is provided already available for you to use in ``/cdtshared/Cohort-2021/HDS-M05/segmentation/MontgomerySet``
- Images are in folder ``/cdtshared/Cohort-2021/HDS-M05/segmentation/MontgomerySet/CXR_png``
- Corresponding masks:
    - Left Mask in ``/cdtshared/Cohort-2021/HDS-M05/segmentation/MontgomerySet/ManualMask/leftMask``
    - Right Mask in ``/cdtshared/Cohort-2021/HDS-M05/segmentation/MontgomerySet/ManualMask/rightMask``
- Data format and size: All are in ``.png`` and of variable size (please resize to 256x256) for your practicals

[2] Transform --> Normalise your data - mean and std (e.g., if color then normalise all three channels)
e.g., torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

[3] Transform --> Always convert data to ToTensor (you can do **step 1, 2 and 3** in one line as done in this tutorial)

[4] New: split your data into **train and validation set**

[4] Make [DataLoaders](https://pytorch.org/docs/stable/data.html): It represents a Python iterable over a dataset

[5] Identify labels 


In [3]:
from torchvision import transforms 
from torch.utils.data import DataLoader

In [5]:
# Preparing transform for step 2 and step 3
# mean = (0.5, 0.5, 0.5)
# std = (0.5, 0.5, 0.5)
# transform = transforms.Compose([
#     transforms.Resize(224),   # Note: here you will resize image to 224 that is input to the AlexNet
#     transforms.ToTensor(),
#     transforms.Normalize(mean, std)
#     ])

from PIL import Image # For reading your custom data images
import os
import glob

# Load data and include prepared transform (Remember to apply same transform to both train 
class mySegmentationData(object):
    def __init__(self, root, transforms = None):
        self.root = root
        self._eval = eval
        self.transforms = transforms
        self.build_dataset()
        
    def build_dataset(self):   
        self.imgs = os.path.join(self.root, "CXR_png")
        self.masks_L = os.path.join(self.root, "ManualMask/leftMask")
        self.masks_R = os.path.join(self.root, "ManualMask/rightMask")
        
        self._images = glob.glob(self.imgs + "/*.png")
        self._labels = glob.glob(self.masks_L  + "/*.png")
#         print(len(self._images))
        
    def __getitem__(self, idx):
    
        img = Image.open(self._images[idx]).convert("L").resize((256,256), resample=0)
        mask = Image.open(self._labels[idx]).convert("RGB").resize((256, 256), resample=0)
        mask = Image.fromarray((np.asarray(mask)[:,:,0].astype('float32')/255.0))
#         Image.fromarray((np.asarray(target)[:,:,0].astype('float32')/255.0).astype('uint8'))
        if self.transforms is not None:
            img = self.transforms(img)
            mask = self.transforms(mask)
   
        return img, mask
    
    def __len__(self):
        return len(self._images)

#### New: You will split train data into train and val set (say 90-10) 
- This step is crucial to identify under- and over-fitting problems 
- Later, we will visualise performance on both train and test online during training (using tensorboard)

In [6]:
# Step: Split between train and valset from the overall trainset
transform = transforms.Compose([ transforms.ToTensor()])
# transforms.Resize((256,256))

import albumentations as A
import albumentations.augmentations.functional as F
from albumentations.pytorch import ToTensorV2

train_transform = A.Compose(
    [
        A.Resize(256, 256),
        A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.5),
        A.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
        A.Normalize(mean=(0.5), std=(0.5)),
        ToTensorV2(),
    ]
)

val_transform = A.Compose(
    [
        A.Resize(256, 256),
        A.Normalize(mean=(0.5), std=(0.5)),
        ToTensorV2(),
    ]
)

trainset = mySegmentationData(root='MontgomerySet/', transforms=train_transform)
valset = mySegmentationData(root='MontgomerySet/', transforms=val_transform)
#  same as you did in classification (here we are doing )
from torch.utils.data.sampler import SubsetRandomSampler
val_percentage = 0.1
num_train = len(trainset)

indices = list(range(num_train))
split = int(np.floor(val_percentage * num_train))

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

print(len(train_sampler))

# Now create data loaders (same as before)
# Now we need to create dataLoaders that will allow to iterate during training
batch_size = 2 # create batch-based on how much memory you have and your data size

traindataloader = DataLoader(trainset, batch_size=batch_size, sampler=train_sampler, num_workers=0)
valdataloader = DataLoader(valset, batch_size=batch_size, sampler=valid_sampler,
            num_workers=0,)

# testdataloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

ModuleNotFoundError: No module named 'albumentations'

In [ ]:
print('Number of training samples:', len(traindataloader))
print('Number of validation samples:', len(valdataloader))
# print('Number of testing samples:', len(testdataloader))

### Look into data

In [ ]:
# function to unnormalise images and using transpose to change order to [H, W, Channel] 
def imshow(img):
    # TODO: unnormalize if needed
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
import matplotlib.pyplot as plt 
# always check the shape of your training data
dataiter = iter(traindataloader)
images, masks = dataiter.next()

# show images 
imshow(torchvision.utils.make_grid(images))
imshow(torchvision.utils.make_grid(masks))
# print labels
# print(' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))


In [ ]:
masks[0].max()


### Create your UNet model
- Please note that your input image size will make difference on your hard-coded feature sizes in FC-layer
- Always be aware of what size input is used, here for convenience we will follow the original paper and reshape image to 224x224x3 

<img src="https://i.stack.imgur.com/EtyQs.png" style="width:800px;height:400px;">
<caption><center> <u>Figure</u>: U-Net architecture for image segmentation.</center></caption>

**Hint**
- You will make an encoder-decoder U-Net architecture 
    - Make a baseblock with --> two convolution with kernel_size 3 and padding 1 with relu 
    - Make another output block with 1 convolution
    - For encoder you will use maxpooling with kernel 2
    - For decoder you will use upsampling using nn.Upsample (scale_factor=2)
    

- You will learn how to group repeated blocks into one while designing U-Net

#### Basic convolution layer -- 2 convs concatenated

In [ ]:
class conv_layer(nn.Module):
    def __init__(self, channel_in, channel_out):
        super().__init__()
        self.conv_l = nn.Sequential(
            nn.Conv2d(channel_in, channel_out, kernel_size = 3, padding = 1),
            DropBlock2D(),
            nn.BatchNorm2d(channel_out),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv_l(x)

In [ ]:
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )   


class UNet(nn.Module):

    def __init__(self, inputchannel, n_class):
        super().__init__()
                
        self.dconv_down1 = double_conv(inputchannel, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)        

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)        
        
        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)
        
        self.conv_last = nn.Conv2d(64, n_class,1)
        
        
    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)   
        
        x = self.dconv_down4(x)
        
        x = self.upsample(x)        
        x = torch.cat([x, conv3], dim=1)
        
        x = self.dconv_up3(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv2], dim=1)       

        x = self.dconv_up2(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv1], dim=1)   
        
        x = self.dconv_up1(x)
        
        out = self.conv_last(x)
        
        return out

### Training your model

### Prepare an optimizer, set learning rate, and you loss function
- Here you will use model.train and use gradients
- Also, you will use criterion to compute loss 
- Compute metric to know how well it is performing
- save them to display mean for each epoch

In [ ]:
# call you model 
from torch import optim
model = UNet(inputchannel=1, n_class=1)
lr = 0.001
# optimiser = optim.Adam(model.parameters(), lr=lr)
# Optimiser
optimiser = torch.optim.Adam(model.parameters(), lr = lr, weight_decay = 1e-8)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# optimiser = torch.optim.RMSprop(model.parameters(), lr = lr, weight_decay = 1e-8, momentum=0.9)
# TODO: you can try with different loss function
# criterion = nn.CrossEntropyLoss()
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

### Prepare accuracy computation to know how your training is going 

[1] Loss function is important to keep track (mostly you minimise it, i.e. it should go down)

[2] Accuracy in classification is important and you want higher accuracy

[3] You can use TensorBoard to visualize both - on new terminal do below

```shell
ssh -L 8889:127.0.0.1:8889 $user@bdicdtvm01.bmrc.ox.ac.uk 
$ ml Anaconda3
$ source activate base

- run your training below and then do this while waiting:
$ tensorboard --logdir runs/ --port 8889
$ http://127.0.0.1:8889/
```

In [ ]:
# Create metrics
def dice(a, b):
    """Calculate dice score for each image in tensor"""
    # a and b are tensors of shape (B, C, H, W)
    # Sum over last two axes (H and W i.e. each image)
    return 2*(a*b).sum(axis=[-2, -1])/(a + b).sum(axis=[-2,-1]).type(torch.float32)

def mask_out(out):
    """Mask tensor/array with 0 threshold"""
    # Need to binarize the output to be able to calculate dice score
    return out > 0

def get_dice_arr(out, label):
    """Get dice score for each image in the batch for each mask seperately"""
    # Output is shape (B, C)
    return dice(mask_out(out), label)

In [ ]:
out[0].max()

In [ ]:
# 4] Run your training loop with optimiser trying to minimise your cost/loss, dont forget to backpropagate your loss
model.to(device)
model.train()
# Tensorboard
from torch.utils.tensorboard import SummaryWriter
# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

# define no. of epochs you want to loop 
epochs = 2
log_interval = 10 # for visualising your iterations 

# New: savining your model depending on your best val score
best_valid_loss = float('inf')
ckptFileName = 'UNet_CKPT_best.pt'
for epoch in range(epochs):
    train_loss, valid_loss, train_dsc,val_dsc  = [], [], [], []
    
    for batch_idx, (data, label) in enumerate(traindataloader):
        # initialise all your gradients to zer

        label = label.to(device)
        optimiser.zero_grad()
        out = model(data.to(device))

        loss = criterion(out, label)
        loss.backward()
        optimiser.step()
        
        # append
        train_loss.append(loss.item())
        acc_1 = get_dice_arr(out, label.to(device))
        
        
        print(acc_1.mean(axis=0).detach().cpu().numpy())
        train_dsc.append(acc_1.mean(axis=0).detach().cpu().numpy())
        
        if (batch_idx % log_interval) == 0:
            print('Train Epoch is: {}, train loss is: {:.6f}'.format(epoch, np.mean(train_loss)))
        
            with torch.no_grad():
                for i, (data, label) in enumerate(valdataloader):
                    data, label = data.to(device), label.to(device)
                    out = model(data)
                    loss = criterion(out, label.to(device))
                    acc_1 = get_dice_arr(out, label.to(device))

                    # append
                    val_dsc.append(acc_1.mean(axis=0).detach().cpu().numpy())
                    valid_loss.append(loss.item())
    
            print('Val Epoch is: {}, val loss is: {:.6f}'.format(epoch, np.mean(valid_loss)))
    
    #New --> save your checkpoint every epoch if best
    if np.mean(valid_loss) < best_valid_loss:
        best_valid_loss = np.mean(valid_loss)
        print('saving my model, improvement in validation loss achieved...')
        torch.save(model.state_dict(), ckptFileName)
        
        
    # every epoch write the loss and accuracy (these you can see plots on tensorboard)        
    writer.add_scalar('UNet/train_loss', np.mean(train_loss), epoch)
#     writer.add_scalar('UNet/train_accuracy', np.mean(train_top1), epoch)
    
    # New --> add plot for your val loss and val accuracy
    writer.add_scalar('UNet/val_loss', np.mean(valid_loss), epoch)
#     writer.add_scalar('UNet/val_accuracy', np.mean(val_top1), epoch)
    
writer.close()

In [ ]:
### TODO: compute the accuracy of your model on validation set --> You are required to use dice similarity coefficient
total = 0
model.eval()
dataiter = iter(valdataloader)
images, masks = dataiter.next()
output = model(images.to(device))


# show images 
# imshow(torchvision.utils.make_grid(images))
# imshow(torchvision.utils.make_grid(output))

output[0].max()
# with torch.no_grad():
#     for data in testdataloader:
#         image, labels = data
#         output = model(image.to(device))
#         _, preds_tensor = torch.max(output, 1)


In [ ]:
# Alternatively you can compute for each class seperately as well 
#(taken from: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testdataloader:
        images, labels = data
        outputs = model(images.to(device))
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label.to(device) == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

#### New => Inference/testing on saved checkpoint
- Load your trained model and apply test on testdataloader 
- If you have checkpoint file (trained weights) you can simply call below for test

In [ ]:
ckptFileName = 'alexNet_CKPT_best.pt'
# load the saved weights
model.load_state_dict(torch.load(ckptFileName))

# Apply testing (same as validation above)
test_preds = []
labels = []
with torch.no_grad():
    model.eval()
    for i, batch in enumerate(testdataloader):
        img, label = batch
        img, label = img.to(device, dtype = torch.float), label.to(device, dtype = torch.long)
        output = model(img)
        output = output.detach().cpu().numpy()
        test_preds.extend(np.argmax(output, 1))
        labels.extend(label.detach().cpu().numpy())

In [ ]:
# --- New => Use scikit-learn to see the confuse matrix/compute accuracy (recall ML-03)
from sklearn.metrics import accuracy_score, confusion_matrix
print('Confusion matrix:')
print(confusion_matrix(labels, test_preds))
print('Accuracy score: %f' % accuracy_score(labels, test_preds))

### Improving your network peerformance

- Train for larger batch size and epochs (longer)
- Add data augmentation provided in transforms (https://pytorch.org/vision/stable/transforms.html) -- Ask tutor if you  are confused 
- Save your training with augmentation as ``my_AlexNet_withAug.pt``
- Tune your hyperparameters and decay rate
- Add your tensorboard outputs for training and validation (loss and accuracy) as image files


#### Exercise: Perform above improvements on CIFAR10 
You can also refer to: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
- You can use this ipython notebook to do this (**Assignment to be submmitted**)
- Due on **Wednesday 3rd November, 2021 (11:59 PM)** (*You will be graded for this exercise*)

<h3>Thanks for completing this lesson!</h3>

Any comments or feedbacks and your solution to exercise, please send to [Sharib Ali](sharib.ali@eng.ox.ac.uk)